# Dependencies and Required Packages

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import seaborn as sns

In [ ]:
from pdfminer.high_level import extract_text
import nltk
import re
import subprocess
nltk.download('stopwords')

In [ ]:
import json

# Generating Text Data From PDF File

In [ ]:
filepath="./resume1.pdf"


In [ ]:
def extract_text_from_pdf(pdf_path):
    text = extract_text(pdf_path)
    return text

txt = extract_text_from_pdf(filepath)


In [ ]:
print(txt)

# Generating Name using NER

In [ ]:
def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names
names = extract_names(txt)
names

In [ ]:
name_candidate = names[0] + ' ' + names[1].split(' ')[0]

# Extracting Phone Number

In [ ]:
PHONE_REG_GH = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')

PHONE_REG_USA = re.compile(r'/^\(?(\d{3})\)?[-]?(\d{3})[-]?(\d{4})$/') 

In [ ]:
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG_GH, resume_text)

    if phone:
        number = ''.join(phone[0])

        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None
 
phone_number_ind = extract_phone_number(txt)
print(phone_number_ind)

In [ ]:
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG_USA, resume_text)

    if phone:
        if resume_text.find(phone) >= 0:
            return phone
    return None
 
phone_number_usa = extract_phone_number(txt)
print(phone_number_usa)

In [ ]:
phone_contact=[]
if phone_number_ind != None:
  phone_contact.append(phone_number_ind)
if phone_number_usa != None:
  phone_contact.append(phone_number_usa)

phone_contact

# Extracting EMAIL ID

In [ ]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

In [ ]:
def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)


In [ ]:
emails = extract_emails(txt)

if emails:
    print(emails)
    

#### General Info Dictionary

In [ ]:
general_dict={ 'Name' : name_candidate.upper(),
              'Email' : emails ,
              'Contact' : phone_contact
    
}

general_dict

# Extracting Skills from the Text

In [ ]:
import pandas as pd
file_skills_domain = pd.read_excel('./ResumeSkill.xlsx')

In [ ]:
file_skills_domain.columns = file_skills_domain.columns.str.strip().str.upper()

In [ ]:
file_skills_domain.columns

In [ ]:
list_domains=[]
for col in file_skills_domain.columns:
  
  file_skills_domain[col] = file_skills_domain[col].str.strip().str.upper()

  if col != 'EDUCATION' :
    list_domains.append('%s' % col)
    globals()['%s' % col]= [x for x in file_skills_domain[col].to_list() if type(x) != float]

In [ ]:
list_domains

In [ ]:
list_skills = []
for i in list_domains:
  list_skills=list_skills + eval(i)
print(list_skills)

### Skills Extraction Loop

In [ ]:
from collections import defaultdict
skills_dict = {}
domain_list = []


def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]

    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]

    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 1, 3)))

    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.upper() in list_skills:
            found_skills.add(token)
    
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.upper() in list_skills:
            found_skills.add(ngram)
            

    for skill in found_skills :
      if skill.upper() not in skills_dict.keys():
          skill = skill.upper()
          cnt = 0
          for i in bigrams_trigrams:
              i = i.upper()
              if skill in i:
                  cnt += 1
                  for j in list_domains:
                    if skill in eval(j):
                      domain_list.append(j)

          print(skill.upper(), ' is repeated ' , cnt, ' times.')
          skills_dict[skill.upper()]= cnt
#     print(list(bigrams_trigrams))
            

In [ ]:
extract_skills(txt)

# print(skills) 

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize = (15,8))

skills_dict
my_df = pd.DataFrame(skills_dict.items())
ax = sns.barplot(x = 0, y = 1, data = my_df)
ax.set(xlabel = 'Skills', ylabel = '% Proficiency', title = 'SKILLS PROFICIENCY OF CANDIDATE ')


In [ ]:
domain_list

from collections import Counter
plt.figure(figsize=(11,7))


new_vals = Counter(domain_list).most_common()
new_vals = new_vals[::-1] #this sorts the list in ascending order

domain_dict = {}
for a, b in new_vals:
    domain_dict[a] = b
 

my_df = pd.DataFrame(domain_dict.items())
ax = sns.barplot(x = 0, y = 1, data = my_df)
ax.set(xlabel = 'Domain', ylabel = '% Score', title = 'FIELD SPECIALITY OF CANDIDATE')


In [ ]:
general_dict["skills"] = skills_dict
general_dict["domain"] = [new_vals[-1][0],new_vals[-2][0]]

### Final Information

In [ ]:
general_dict

In [ ]:
from pandas import json_normalize 
skill_df = json_normalize(general_dict['skills'])
num = skill_df.sum(axis = 1)[0]

list_details = []
for skill_x in skill_df.columns:
    list_details.append({'doc':filepath ,'Name' : name_candidate.upper(),
              'email' : emails[0] ,
              'contact' : phone_contact[0],
              'domain':[new_vals[-1][0][5:],new_vals[-2][0][5:]],
              'skills':skill_x ,
              'normalised_count':round((skill_df[skill_x][0]*100)/num,2),
              'total_skills':list(skill_df.columns)
              })
# pd.DataFrame(list_details)

### Checking Similarity to Job Description

In [ ]:
import docx2txt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

job_description = docx2txt.process("./job_desc.docx")
resume = docx2txt.process("./resume1.docx")

text = [resume, job_description]

# Calculate similarity score 
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)
print("\Similarity Score: ")
print(cosine_similarity(count_matrix))

# Count match percentage
match_percentage = cosine_similarity(count_matrix)[0][1] * 100
match_percentage = round(match_percentage, 2)
print()
print("The match percentage of the resume is ", match_percentage,"%")
